## 프로그램 언어의 로직을 그룹핑 하는 방식

- ### 절차형 방법에서는 코드의 묶음을 하나의  프로시저 또는 서브루틴
- ### 함수형 방법에서는 코드의 묶음을 함수
- ### 객체지향 방법에서는 코드의 묶음을 클래스(변수와 함수를 하나로 묶음)

## 함수

- ### 순수함수 : 외부에 영향 X
- ### 비순수함수 : 외부에 영향 O
- ### FP의 중요한 2가지 : 불변(thread-safe), 순수함수
- ## 선언과 정의 
   * ### 선언 : 몸통 X
   * ### 정의 : 어떤 문장들을 묶는 것 / 명령문들을 그룹으로 묶는 것 -> Scope생김 -> Map이 있음 -> locals()로 접근 

In [4]:
## 함수 호출하기 : ()

def do_nothing():
    pass # 아무것도 하지않는다는 것을 의미

do_nothing()
print(do_nothing())

None


In [3]:
## 인수와 매개변수

def echo(anything):
    return anything + ' ' + anything

echo('run')

'run run'

In [19]:
## 유용한 None
# None 은 아무것도 없다는 것
# None과 False는 다르다 

## 위치 인수 
# 위치 인수의 단점 : 인수의 각 위치에 대한 의미를 알아야 한다는 것

## 키워드 인수 (이름=값)
# 매개변수에 상응하는 이름을 인수에 지정하는 것 

## 기본 매개변수 값 지정하기
def menu(wine,entree,dessert='pudding'): # dessert 인수를 제공하지 않으면 기본인수 대입
    return {'wine':wine,'entree':entree, 'dessert':dessert}

# 기본값이 가변 객체인 경우, 해당 객체는 모든 함수 호출 간에 공유
def buggy(arg,result=[]):
    result.append(arg)
    print(result)
    
buggy('a') # ['a']
buggy('b') # ['a', 'b']

# 해결방법
def buggy(arg, result=None):
    if result is None:
        result = []
    result.append(arg)
    print(result)

# 함수 호출
buggy('a')  # 출력: ['a']
buggy('b')  # 출력: ['b']

## 위치 인수 분해하기/모으기:  * 
#  -> 매개변수에서 위치 인수 변수를 튜플로 묶음 (가변인수) / 함수 호출 또는 정의에서만 * 구문을 사용할 수 있음
def print_args(*args):
    print('Positional tuple:',args)
    
print_args()
print_args(3,2,1,'wait!','2') # Positional tuple: (3, 2, 1, 'wait!', '2')
args = (2,5,1)
print_args(args) # Positional tuple: ((2, 5, 1),)
print_args(*args) # Positional tuple: (2, 5, 1)  -> args 튜플을 풀어서 각각의 요소를 개별적인 인자로 전달

## 키워드(이름=값) 인수 분해하기/모으기 : **
# -> 키워드 인수를 딕셔너리로 묶음  / 함수 호출 또는 정의에서만 * 구문을 사용할 수 있음
def print_kwargs(**kwargs):
    print('Keyword arguments:',kwargs)

print_kwargs(wine='merlot',entree='mutton',dessert='marcaronn') # Keyword arguments: {'wine': 'merlot', 'entree': 'mutton', 'dessert': 'marcaronn'}


## 키워드(이름=값) 전용 인수
def print_data(data,*,start=0,end=100): # * 이후에 정의된 start와 end는 키워드 전용 인자가 되어야함
    for value in (data[start:end]):
        print(value)
        
data = [1,2,3,4,5,6]

print_data(data,start=2)
print_data(data,end=2)


## 독스트링 (docstring)
# 함수 바디 시작 부분에 문자열을 포함시켜 함수 정의에 문서를 붙일 수 있는 것
def echo(anything):
    'echo returns its input argument'
    return anything

help(echo) 
# Help on function echo in module __main__:

# echo(anything)
# echo returns its input argument

print(echo.__doc__) # echo returns its input argument




['a']
['a', 'b']
['a']
['b']
Positional tuple: ()
Positional tuple: (3, 2, 1, 'wait!', '2')
Positional tuple: ((2, 5, 1),)
Positional tuple: (2, 5, 1)
Keyword arguments: {'wine': 'merlot', 'entree': 'mutton', 'dessert': 'marcaronn'}
3
4
5
6
1
2
Help on function echo in module __main__:

echo(anything)
    echo returns its input argument

echo returns its input argument


In [10]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:
    def __init__(self,customer,cart,promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    def total(self):
        if not hasattr(self,'_total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due:{: 2f}>'
        return fmt.format(self.total(),self.due())

class Promotion(ABC): # 전략:추상 베이스 클래스
    @abstractmethod
    def discount(self,order):
        """할인액을 구체적인 숫자로 반환한다"""

class FidelityPromo(Promotion):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self,order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self,order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 10% 할인 적용"""

    def discount(self,order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total()* .07
        return 0

joe = Customer('John Doe',0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple',10, 1.5),
        LineItem('watermellon',5,5.0)]
o1 = Order(joe,cart,FidelityPromo())
o2 = Order(ann,cart,FidelityPromo())
print(o1,o2)


<Order total: 42.00 due: 42.000000> <Order total: 42.00 due: 39.900000>


## 일등 함수 

### 함수도 1급 객체
### 함수도 다른 객체처럼 클래스를 가지고 있다.
### 함수도 다른 객체처럼 변수에 할당도 가능하다. 즉 매개변수로 전달이 가능하다.


In [12]:
def answer():
    print(42)
    
answer() # 42

def run_something(func):
    func()
    
run_something(answer()) # 42

42
42


TypeError: 'NoneType' object is not callable

### 파이썬에서 괄호 ()는 함수를 호출한다는 의믜
   * ### 괄호가 없으면 함수를 다른 모든 객체와 마찬가지로 간주 why? 파이썬에서 모든 것은 객체이기 때문



In [20]:
type(run_something)

def sum_args(*args):
    return sum(args)

sum_args(1,2,3)

def rum_with_positional_args(func,*args):
    return func(*args)

rum_with_positional_args(sum_args,1,2,3)

6

## 내부 함수
### 내부함수는 반복문이나 코드 중복을 피하고자 또 다른 함수 내에 어떤 복잡한 작업을 한번 이상 수행할때 유용

## 클로저 -> Scope를 묶은 것 (Python에는 접근제어자가 없어서 캡슐화가 안되는데 클로저가 해결)
### 다른 함수에 의해 동적으로 생성됨
### 외부 함수로부터 생성된 변수값을 변경하고, 저장할 수 있는 함수 

In [31]:
def make_counter():
    count = 0 # cv

    def counter():
        nonlocal count
        count += 1
        return count

    return counter

counter = make_counter()
print(counter())  # 출력: 1
print(counter())  # 출력: 2

def knights2(saying):
    def inner2():
        return "We are the knights who say: '%s'"%saying
    return inner2 # inner2함수를 동적으로 생성

a = knights2('Duck')
b = knights2('Hasenpfeffer')

print(type(a),end=", ")
print(type(b)) # <class 'function'>, <class 'function'>

print(a,b) 
# <function knights2.<locals>.inner2 at 0x000001755B339B20> <function knights2.<locals>.inner2 at 0x000001755B33A520>

# chatGPT에게 질문 : 실행중에도 inner2 함수의 네임스페이스에는 saying이 존재하는게 아니라 접근하는거지?
#실행 중에도 inner2 함수의 네임스페이스에는 saying이 직접적으로 존재하지 않지만, 클로저를 통해 saying 변수에 접근할 수 있습니다. 이는 클로저의 중요한 특징으로, 내부 함수가 외부 함수의 변수에 접근할 수 있게 해줍니다.

# 일반적으로 함수 내에서 선언된 변수는 함수가 실행되는 동안에만 존재하고, 함수 실행이 끝나면 사라집니다. 그러나 클로저를 사용하면 함수가 종료된 후에도 그 변수의 값을 유지하면서 접근할 수 있습니다.

def make_counter():
    count = 0

    def increment():
        nonlocal count
        count += 1
        return count

    return increment

counter = make_counter()
print(counter())  # 출력: 1
print(counter())  # 출력: 2
print(counter())  # 출력: 3


1
2
<class 'function'>, <class 'function'>
<function knights2.<locals>.inner2 at 0x000001755C00B6A0> <function knights2.<locals>.inner2 at 0x000001755C00A160>
1
2
3
4
5
6


## 익명 함수 : lamda
### 단일 문장으로 표현되는 익명 함수
### lambda 키워도 지정
### 콜론 좌측은 매개변수 우측은 표현식 지정
### 문장을 표현식에 작성하면 예외를 발생시킴

## 제너레이터
### 시퀀스를 생성하는 객체 ex) range() 함수

## 제너레이터 함수
###  yield 문을 사용하면 함수의 실행 상태를 기억하고, 다음 호출 시 저장된 상태에서 다시 실행을 시작할 수 있습니다.


In [45]:
def my_range(first=0,last=10,step=1):
    number = first
    while number < last:
        yield number
        number += step

a = my_range(1,5)
b = my_range(1,6)
for x in a:
    print(x)

for i in b:
    print(i)

for x1 in a:
    print(x1) # 순회를 마친 제너레이터를 다시 순회하면 아무것도 반환하지 않음 -> java의 iterator와 유사


1
2
3
4
1
2
3
4
5


## 데커레이터

### 특정 함수나 클래스로 기존 함수나 클래스의 기능을 추가하는 경우 사용
### 보통 함수에 함수를 전달해서 새로운 기능을 처리 : 합성함수 처리

### 데코레이터 함수 내에 내부 함수를 정의해서 새로운 기능 추가
   * 함수를 전달 받아 지역변수로 처리
  * 내부 함수를 반환한다
    * 내부 함수 내에서 저장된 함수를 실행한다   
      
### 데코레이터 단순 표기법
   * 기존 함수에 @ 기호와 데코레이터 함수를 지정한다
  * 함수를 실행할 때는 실행함수 이름으로 실행
    * 내부 함수가 실행함수 이름으로 반환되어 처리

In [92]:
def dec(func):
    print('a')
    def new_func():
        print('A')
        return func()
    return new_func

def add():
    return 10

@dec
def _add():
    return 10
# add = dec(add)
# add()
_add()

a
A


10

### 데코레이터 실행함수 인자 전달 하기
  * 실행함수 저장과 실행함수의 인자를 분리
  * 내부 함수는 실행함수의 인자를 받아서 저장된 실행함수를 실행
 

In [78]:
def dec_1(func):
    print('a')
    def new_func(*args,**kwargs):
        print('A')
        return func(*args,**kwargs)
    return new_func

@dec_1
def add_1(x,y):
    return x+y



a


In [79]:
add_1(100,200)

A


300

In [83]:
def add(x,y):
    result = add(2,5)
    return result

print(add)

<function add at 0x000001755C1A2200>


## 데코레이터 실행순서
 * 데코레이터는 여러 개를 실행함수에 연결할 수 있다.
 * 데코레이터 순서는 밑에 정의한 것부터 차례대로 구성된다.

In [87]:
def a(func):
    print("a")
    def new_func():
        print('A')
        func()
    return new_func


def b(func):
    print("b")
    def new_func():
        print('B')
        func()
    return new_func


def c(func):
    print("c")
    def new_func():
        print('C')
        func()
    return new_func


@a
@b
@c # <- 우선 적용
def func():
    pass
func()

c
b
a
A
B
C


### 데코레이터 적용 순서
 * 로딩시 자동으로 데코레이터가 구성된다
### 실행순서
 * 실행순서는 제일 위에 정의한 데코레이터부터 순차적으로 실행된다

In [104]:
def func_1():
    pass

func_2 = a(b(c(func_1))) # 데코레이터 문법 적용시키지 않고 동일한 함수를 생성
func_2()

c
b
a
A
B
C


In [102]:
def document_it(func):
    def new_function(*args,**kwargs):
        print('Running function:',func.__name__)
        return func(*args,**kwargs)
    return new_function

def square_it(func):
    def new_function(*args,**kwargs):
        result = func(*args,**kwargs)
        return result*result
    return new_function

@document_it
@square_it
def add_ints(a,b):
    return a+b

add_ints(3,5)

Running function: new_function


64